## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import os
import pandas as pd
import requests
import gmaps
import numpy as np

In [2]:
# Import API key
pwd = os.getcwd()
td = os.path.join(pwd,"..")
os.chdir(td)

from config import g_key
os.chdir(pwd)

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
file_to_load = os.path.join("..","Weather_Database","WeatherPy_database.csv")
city_data_df = pd.read_csv(file_to_load)
city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Nikolskoye,RU,59.7035,30.7861,-12.00,72,75,7.00,snow
1,Albany,US,42.6001,-73.9662,3.00,35,75,6.17,broken clouds
2,Olinda,BR,-8.0089,-34.8553,28.00,61,40,4.12,scattered clouds
3,Klaksvik,FO,62.2266,-6.5890,1.00,64,40,9.77,scattered clouds
4,Flinders,AU,-34.5833,150.8552,17.78,86,0,0.45,clear sky


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 25
What is the maximum temperature you would like for your trip? 35


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,Olinda,BR,-8.0089,-34.8553,28.00,61,40,4.12,scattered clouds
9,Caravelas,BR,-17.7125,-39.2481,25.74,89,100,4.01,overcast clouds
10,Atuona,PF,-9.8000,-139.0333,26.74,78,100,10.42,overcast clouds
11,Avarua,CK,-21.2078,-159.7750,29.00,74,45,3.60,light rain
13,Carnarvon,AU,-24.8667,113.6333,27.00,100,40,5.14,scattered clouds
20,Bintulu,MY,3.1667,113.0333,25.00,100,40,0.51,light rain
22,Rikitea,PF,-23.1203,-134.9692,25.47,73,100,5.24,light rain
27,Viedma,AR,-40.8135,-62.9967,25.00,46,0,0.45,clear sky
35,Ternate,ID,0.8000,127.4000,26.12,81,100,0.40,overcast clouds
37,Bubaque,GW,11.2833,-15.8333,27.85,57,45,5.76,scattered clouds


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City                   139
Country                138
Lat                    139
Lng                    139
Max Temp               139
Humidity               139
Cloudiness             139
Wind Speed             139
Current Description    139
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.replace("",np.nan).dropna()
clean_df.count()

City                   138
Country                138
Lat                    138
Lng                    138
Max Temp               138
Humidity               138
Cloudiness             138
Wind Speed             138
Current Description    138
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Olinda,BR,28.00,scattered clouds,-8.0089,-34.8553,
9,Caravelas,BR,25.74,overcast clouds,-17.7125,-39.2481,
10,Atuona,PF,26.74,overcast clouds,-9.8000,-139.0333,
11,Avarua,CK,29.00,light rain,-21.2078,-159.7750,
13,Carnarvon,AU,27.00,scattered clouds,-24.8667,113.6333,
20,Bintulu,MY,25.00,light rain,3.1667,113.0333,
22,Rikitea,PF,25.47,light rain,-23.1203,-134.9692,
27,Viedma,AR,25.00,clear sky,-40.8135,-62.9967,
35,Ternate,ID,26.12,overcast clouds,0.8000,127.4000,
37,Bubaque,GW,27.85,scattered clouds,11.2833,-15.8333,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except:
        print(f"Hotel not found in {row['City']}... skipping.")        

Hotel not found in Lagos... skipping.
Hotel not found in Tucuma... skipping.
Hotel not found in Gaoua... skipping.
Hotel not found in Filingue... skipping.
Hotel not found in Saint-Louis... skipping.
Hotel not found in Aripuana... skipping.
Hotel not found in Merauke... skipping.
Hotel not found in Sembe... skipping.
Hotel not found in Bafoulabe... skipping.
Hotel not found in Thinadhoo... skipping.
Hotel not found in Yaan... skipping.
Hotel not found in Bonfim... skipping.


In [10]:
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Olinda,BR,28.00,scattered clouds,-8.0089,-34.8553,Flat Quatro Rodas Olinda
9,Caravelas,BR,25.74,overcast clouds,-17.7125,-39.2481,Pousada dos Navegantes
10,Atuona,PF,26.74,overcast clouds,-9.8000,-139.0333,Villa Enata
11,Avarua,CK,29.00,light rain,-21.2078,-159.7750,Paradise Inn
13,Carnarvon,AU,27.00,scattered clouds,-24.8667,113.6333,Hospitality Carnarvon


In [11]:
hotel_df.count()

City                   138
Country                138
Max Temp               138
Current Description    138
Lat                    138
Lng                    138
Hotel Name             138
dtype: int64

In [12]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.replace("",np.nan).dropna()
clean_hotel_df.count()

City                   126
Country                126
Max Temp               126
Current Description    126
Lat                    126
Lng                    126
Hotel Name             126
dtype: int64

In [13]:
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Olinda,BR,28.00,scattered clouds,-8.0089,-34.8553,Flat Quatro Rodas Olinda
9,Caravelas,BR,25.74,overcast clouds,-17.7125,-39.2481,Pousada dos Navegantes
10,Atuona,PF,26.74,overcast clouds,-9.8000,-139.0333,Villa Enata
11,Avarua,CK,29.00,light rain,-21.2078,-159.7750,Paradise Inn
13,Carnarvon,AU,27.00,scattered clouds,-24.8667,113.6333,Hospitality Carnarvon


In [14]:
# 8a. Create the output File (CSV)

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv("WeatherPy_vacation.csv", index_label="City_ID")

In [15]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °C</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [16]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))